In [2]:
import json
import pandas as pd
import os

# Data Preprocessing - Two Dataset

In [38]:
# Data Preprocessing Benchmarking Large Language Models for News Summarization

file_name = "/Users/lujun.li/projects/causallm-github/causalllm/resource/summarization/benchmark_llm_summarization/pairwise_evaluation_results.json"
with open(file_name, "r") as file:
    pairwise_evaluation_results = json.load(file)
pairwise_evaluation_results_df = pd.DataFrame(pairwise_evaluation_results)

file_name = "/Users/lujun.li/projects/causallm-github/causalllm/resource/summarization/benchmark_llm_summarization/likert_evaluation_results.json"
with open(file_name, "r") as file:
    likert_evaluation_results = json.load(file)

likert_evaluation_results_df = pd.DataFrame(likert_evaluation_results)

file_name = "/Users/lujun.li/projects/causallm-github/causalllm/resource/summarization/benchmark_llm_summarization/writer_summaries.json"
with open(file_name, "r") as file:
    writer_summaries_results = json.load(file)
writer_summaries_results_df = pd.DataFrame(writer_summaries_results)

In [39]:
system_message = "You are a writer who is good at summarizing articles."
USER_MESSAGE = "Summarize the following article in 1 to 3 sentences."

initial_prompt = """
Role: {system_message}
Instruction: {USER_MESSAGE}
Article:
{article}
"""

mixtral_instruct_initial_prompt = """<s>[INST]
{system_message}
Instruction: {USER_MESSAGE}
Article: 
{article}
[/INST] 
</s>"""

llama2_chat_initial_prompt = """
<s>[INST] <<SYS>>
{system_message}
<</SYS>>
Instruction: {USER_MESSAGE}
Article:
{article}
[/INST]
"""

llama3_chat_initial_prompt = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{system_message}
<|eot_id|><|start_header_id|>user<|end_header_id|>
Instruction: {USER_MESSAGE}
Article:
{article}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

In [40]:
# columns = ["article", "model", "summary", "dataset"]
# likert_evaluation_results_df_drop = likert_evaluation_results_df[
#     likert_evaluation_results_df["model"] == "reference"
# ][columns].drop_duplicates()
# likert_evaluation_results_df_drop.shape

# likert_evaluation_results_df_drop["origin"] = (
#     "benchmark_llm_summarization_human_included"
# )
# columns = ["article_text", "writer_summary", "text-davinci-002_summary"]
# pairwise_evaluation_results_df_shortened = pairwise_evaluation_results_df[columns]
# pairwise_evaluation_results_df_shortened.rename(
#     columns={"article_text": "article"}, inplace=True
# )
# pairwise_evaluation_results_df_shortened.iloc[0].article

In [41]:
columns = ["article", "summary"]
writer_summaries_results_df_shortened = writer_summaries_results_df[columns]

In [42]:
writer_summaries_results_df_shortened.rename(
    columns={"summary": "humman_summary"}, inplace=True
)

/var/folders/k6/w4zkqz352hzf1fs2pc1tqjf88vhly_/T/ipykernel_61413/2317706942.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  writer_summaries_results_df_shortened.rename(


In [43]:
writer_summaries_results_df_shortened

,article,humman_summary
0,Nick Scholfield is lined up to ride Spring Hee...,Nick Schofield is riding Spring Heeled in the ...
1,Nick Scholfield is lined up to ride Spring Hee...,Preparation is taking place for a horse ridin...
2,Nick Scholfield is lined up to ride Spring Hee...,Nick Scholfield will travel to Ireland and is ...
3,Dr Mehmet Oz's fellow faculty members at Colum...,Faculty at Columbia University have written an...
4,Dr Mehmet Oz's fellow faculty members at Colum...,"Celebrity doctor, Mehmet Oz, is being attacked..."
...,...,...
297,An inquest into an IRA massacre of 10 Protesta...,A 40-year inquest has been halted due to the l...
298,"Rare drawings by Leonardo da Vinci, which are ...",Rare drawings by Leonardo da Vinci have gone o...
299,"BANGKOK, Thailand (CNN) -- A Bangkok Airways p...",A Bangkok Airways plane crashed killing the pi...
300,Maverick Tottenham forward Emmanuel Adebayor h...,The 31-year-old Emmanual Adebayor took to Twit...


In [44]:
from langchain import PromptTemplate

initial_prompt_template = PromptTemplate.from_template(initial_prompt)
llama2_chat_initial_prompt_template = PromptTemplate.from_template(
    llama2_chat_initial_prompt
)
llama3_chat_initial_prompt_template = PromptTemplate.from_template(
    llama3_chat_initial_prompt
)
mixtral_instruct_initial_prompt_template = PromptTemplate.from_template(
    mixtral_instruct_initial_prompt
)


def generate_initial_prompts(row):
    article = row["article"]
    initial_prompt = initial_prompt_template.format(article=article, system_message=system_message, USER_MESSAGE=USER_MESSAGE)
    llama2_chat_initial_prompt = llama2_chat_initial_prompt_template.format(
        article=article, system_message=system_message, USER_MESSAGE=USER_MESSAGE
    )
    llama3_chat_initial_prompt = llama3_chat_initial_prompt_template.format(
        article=article, system_message=system_message, USER_MESSAGE=USER_MESSAGE
    )
    mixtral_instruct_initial_prompt = mixtral_instruct_initial_prompt_template.format(
        article=article, system_message=system_message, USER_MESSAGE=USER_MESSAGE
    )
    row["initial_prompt"] = initial_prompt
    row["llama2_chat_initial_prompt"] = llama2_chat_initial_prompt
    row["llama3_chat_initial_prompt"] = llama3_chat_initial_prompt
    row["mixtral_instruct_initial_prompt"] = mixtral_instruct_initial_prompt
    row["SYSTEM_MESSAGE"] = system_message
    row["USER_MESSAGE"] = USER_MESSAGE
    return row


writer_summaries_results_df_shortened = writer_summaries_results_df_shortened.apply(
    generate_initial_prompts, axis=1
)

In [45]:
writer_summaries_results_df_shortened.shape

(302, 8)

In [46]:
writer_summaries_results_df_shortened.to_csv("summary.csv")

In [48]:
import numpy as np

temperatures = np.arange(0.1, 2.0, 0.3)
temperature_df = pd.DataFrame({"Temperature": temperatures})
merged_df = temperature_df.merge(writer_summaries_results_df_shortened, how="cross")

In [50]:
merged_df.to_csv("Summary_experiment_prompt.csv")